# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display
import toml
import gspread
from google.oauth2.service_account import Credentials
from auxiliar.arrumar_bases import *

def get_sheetdata(sheet_name):
    # Load credentials from the .streamlit/secrets.toml file
    secrets = toml.load(".streamlit/secrets.toml")
    creds_info = secrets["connections"]["gsheets"]

    # Define scopes for accessing Google Sheets and Drive
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    # Create credentials and authorize the gspread client
    creds = Credentials.from_service_account_info(creds_info, scopes=scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet using the URL from the toml file
    spreadsheet = client.open_by_url(creds_info["spreadsheet"])

    # Select the first worksheet and fetch all records
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()

    df = pd.DataFrame(data)

    return df
    
vmb_path = "assets/venda-mensal-bruta_colab-bi_2025-02-11.xlsx"
agendamento_path = "assets/agendamentos_colab-bi_2025-02-12 (1).xlsx"

venda_mensal_df = pd.read_excel(vmb_path)
agendamentos_df = pd.read_excel(agendamento_path)
procedimentos_df = get_sheetdata("procedimentos_padronizados")
prestadora_df = get_sheetdata("base_prestadoras")
comissao_df = get_sheetdata("comissoes")
tipo_prestadora_df = get_sheetdata("tipo_prestadora")



In [ ]:
# Todas as bases

base_compilada = criar_base_compilada(agendamentos_df,venda_mensal_df,procedimentos_df,prestadora_df,comissao_df,tipo_prestadora_df)
base_lavieen = criar_base_lavieen(base_compilada)
base_comissoes = criar_comissoes(base_compilada)
base_avaliacoes = cria_avaliacoes(base_compilada,"mensal")
revenda_df = cria_base_revenda(venda_mensal_df)

# Visualizar

In [3]:
base_compilada = criar_base_compilada(agendamentos_df,venda_mensal_df,procedimentos_df,prestadora_df,comissao_df,tipo_prestadora_df)
base_lavieen = criar_base_lavieen(base_compilada)
base_comissoes = criar_comissoes(base_compilada)
base_avaliacoes = cria_avaliacoes(base_compilada,"mensal")
revenda_df = cria_base_revenda(venda_mensal_df)


base_com_lavieen = juntar_bases(base_comissoes,base_avaliacoes,revenda_df,base_lavieen)

/home/ciroshia/Downloads/github/comissao/auxiliar/arrumar_bases.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenda_df["Unidade"] = revenda_df["Unidade"].str.normalize("NFKC").str.strip().str.lower()
/home/ciroshia/Downloads/github/comissao/auxiliar/arrumar_bases.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenda_df["nome_prestadora"] = revenda_df["nome_prestadora"].str.normalize("NFKC").str.strip().str.lower()


In [6]:
base_lavieen_filtrada = base_lavieen.loc[base_lavieen["procedimento_padronizado"].str.contains("lavieen",case=False,na=False)]
base_lavieen_groupby = base_lavieen_filtrada.groupby(["nome_prestadora","Unidade"]).agg(quantidade_lavieen=('Data', 'nunique')).reset_index()

comissao_lavieen = 500
base_lavieen_groupby["valor_lavieen"] = comissao_lavieen * base_lavieen_groupby["quantidade_lavieen"]

base_lavieen_groupby

,nome_prestadora,Unidade,quantidade_lavieen,valor_lavieen
0,amanda souza faioli,guarulhos,2,1000
1,amanda souza faioli,tucuruvi,2,1000
2,anna júlia santos dancini,londrina,3,1500
3,bruna cassiolatto basiotti,mooca,3,1500
4,claudenice da silva girao,tucuruvi,1,500
5,elaine teixeira elizeu,tucuruvi,2,1000
6,giulia de oliveira quintino,guarulhos,1,500
7,giulia de oliveira quintino,ipiranga,1,500
8,gleiciane dos santos rocha,osasco,1,500
9,isabela serafim dias,londrina,3,1500
